# Aplicações práticas

# 2.4 Exercício 4: Sexo e Comportamento

## A. Enunciado 

Uma amostra aleatória de 3182 pessoas de ambos os sexos (1 Mulher, 2 Homem) foi seguida ao longo de 3 anos, com vista a apurar o seu comportamento planeado de agressividade (1. Agressivo, 2, Assertivo). Observaram-se 1584 mulheres e 1598 Homens. Nas mulheres 1110 tinham comportamento planeado de agressividade, enquanto nos homens esse valor baixou para 477. Ambas as variáveis são nominais, por serem apenas definidas pelo seu nome. O estudo é prospetivo de coorte e os dados observados constam da linha Count da Tabela 1, do ficheiro "cap02_exerc.4_Sexo_Comportamento.sav" e e organizados em um dicionário em Python.

# Resolução em Python

## Carregando Bibliotecas

In [1]:
import pandas as pd  # Fornece DataFrames e Series para manipulação de dados em Python, facilitando operações como leitura, escrita, e manipulação de estruturas de dados tabulares.
from scipy.stats import chi2  # Realiza cálculos estatísticos relacionados à distribuição qui-quadrado

## Funções Personalizadas

In [2]:
!pip install socialdataanalysis
!pip install --upgrade socialdataanalysis
from socialdataanalysis import analisar_independencia_variaveis_tabela_contingencia
from socialdataanalysis import gerar_tabela_contingencia
from socialdataanalysis import complementar_tabela_contingencia_com_analise_estatistica
from socialdataanalysis import calcular_odds_ratio_razao_risco_discrepancia

## Dados fornecidos

In [3]:
# Dicionário com dados categorizados representando grupos e suas frequências
dados = {
    'Sexo': [1, 1, 2, 2],
    'Comportamento': [1, 2, 1, 2],
    'Freq': [1101, 483, 477, 1121]
}

# Criando o DataFrame a partir do dicionário
dados = pd.DataFrame(dados)

# Convertendo as colunas para inteiros, se necessário
dados = dados.astype(int)

# Visualizando o dataframe
display(dados)

Sexo  Comportamento  Freq
0     1              1  1101
1     1              2   483
2     2              1   477
3     2              2  1121

## Tabela 1: Tabela de Contingência

In [4]:
# Obter automaticamente os nomes dos grupos e da coluna de frequência
grupos = dados.columns[:-1].tolist()  # Todos exceto a última coluna
categorias = {
    grupos[0]: ['1. Mulher', '2. Homem',],
    grupos[1]: ['1. Agressivo', '2. Assertivo'],
}

tabela_contingencia = gerar_tabela_contingencia(dados, grupos, categorias)

display(tabela_contingencia)

Comportamento  1. Agressivo  2. Assertivo
Sexo                                     
1. Mulher              1101           483
2. Homem                477          1121

## Tabela de Contingência com Cálculos

In [5]:
# Função própria complementar_tabela_contingencia_com_analise_estatistica
combined_df = complementar_tabela_contingencia_com_analise_estatistica(tabela_contingencia)

# Visualizando resultado
display(combined_df)

Comportamento                    1. Agressivo 2. Assertivo   Total
Sexo                                                              
1. Mulher Count                        1101.0        483.0  1584.0
          Expected Count                785.5        798.5  1584.0
          % within Sexo                  69.5         30.5   100.0
          % within Comportamento         69.8         30.1    49.8
          % of Total                     34.6         15.2    49.8
          Standardized Residual          11.3        -11.2        
          Adjusted Residual              22.4        -22.4        
2. Homem  Count                         477.0       1121.0  1598.0
          Expected Count                792.5        805.5  1598.0
          % within Sexo                  29.8         70.2   100.0
          % within Comportamento         30.2         69.9    50.2
          % of Total                     15.0         35.2    50.2
          Standardized Residual         -11.2         11.1        
          Adjusted Residual             -22.4         22.4        
Total     Count                        1578.0       1604.0  3182.0
          Expected Count               1578.0       1604.0  3182.0
          % within Sexo                  49.6         50.4   100.0
          % within Comportamento        100.0        100.0   100.0
          % of Total                     49.6         50.4   100.0

<font color="blue">    
    
### 2.1.1. Probabilidades e Teorema de Bayes
</font>

<font color="blue">    
    
#### a) Probabilidade de ser mulher.
</font>

In [13]:
## PROBABILIDADES MARGINAIS ##

# Calculando as Probabilidades Marginais para o Grupo 1
prob_marg_grupo1 = tabela_contingencia.sum(axis=1) / tabela_contingencia.values.sum()
# Convertendo em um DataFrame
prob_marg_grupo1 = prob_marg_grupo1.to_frame(name='')

# Exibindo os DataFrames
print(f"Probabilidades Marginais de {grupos[0]}")
display(prob_marg_grupo1.round(3))

# Impressão de configuração específica
evento = categorias[grupos[0]][0]
print(f"P({evento}) = {prob_marg_grupo1.loc[evento, '']:.3f}")

Probabilidades Marginais de Sexo


Sexo            
1. Mulher  0.498
2. Homem   0.502

P(1. Mulher) = 0.498


<font color="blue">

#### b) Probabilidade de ser muher sabendo que teve um comportamento planeado de agressividade. Compare o resultado com o obtido no ponto anterior.
</font>

In [12]:
## PROBABILIDADES CONDICIONADAS ##

# Calculando P(grupo1|grupo2)
prob_cond_grupo1_dado_grupo2 = tabela_contingencia.div(tabela_contingencia.sum(axis=0), axis=1)

# Exibindo as probabilidades condicionadas com arredondamento para 3 casas decimais
print(f"Probabilidade Condicionada P({grupos[0]}|{grupos[1]}):")
display(prob_cond_grupo1_dado_grupo2.round(3))

# Impressão de configuração específica
evento = categorias[grupos[0]][0]
condicao = categorias[grupos[1]][0]
print(f"P({evento}|{condicao}) = {prob_cond_grupo1_dado_grupo2.loc[evento, condicao]:.3f}")

Probabilidade Condicionada P(Sexo|Comportamento):


Comportamento  1. Agressivo  2. Assertivo
Sexo                                     
1. Mulher             0.698         0.301
2. Homem              0.302         0.699

P(1. Mulher|1. Agressivo) = 0.698


<font color="blue">   

### 2.4.2. Teste do rácio da Verosimilhança 
</font>

<font color="blue">

#### c) Pressupostos e hipóteses do teste.
</font>

In [8]:
analisar_independencia_variaveis_tabela_contingencia(tabela_contingencia, 
                                                     mostrar_pearson=True, 
                                                     mostrar_continuity=True, 
                                                     mostrar_likelihood=True, 
                                                     mostrar_fisher=False)

print("\nHipóteses:")
print(f"\nH0: Não há associação entre {grupos[0]} e {grupos[1]}; as variáveis são independentes.")
print(f"Ha: Há uma associação entre {grupos[0]} e {grupos[1]}; as variáveis não são independentes.")

Value df Asymp. Sig. (2-sided)
Test                                                  
Pearson Chi-Square     500.47  1                 0.000
Continuity Correction  498.88  1                 0.000
Likelihood Ratio       514.48  1                 0.000
N. of Valid Cases        3182

a. 0 cells (0.00%) have expected count less than 5. The minimum expected count is 785.53.

Hipóteses:

H0: Não há associação entre Sexo e Comportamento; as variáveis são independentes.
Ha: Há uma associação entre Sexo e Comportamento; as variáveis não são independentes.


<font color="blue">

#### d) Para um erro do analista de 0.01, determine RA e RC para o valor do teste e comente a significância da associação.
</font>

In [11]:
# Definindo o nível de significância (erro do analista)
alpha = 0.01  # Erro do analista é 1% ou 0.01

# Calculando a área até a fronteira, que é 1 - alpha para um teste de cauda única
nivel_confianca  = 1 - alpha

# Grau de liberdade para o teste
df = 1

# Calculando o valor crítico do qui-quadrado (RC)
valor_critico = chi2.ppf(nivel_confianca, df)

# RA (Região de Aceitação) é tudo aquilo abaixo do valor crítico
# RC (Região Crítica) é tudo aquilo acima do valor crítico
print(f"Região de Aceitação (RA): [0; {valor_critico:.3f}[")
print(f"Região Crítica (RC): [{valor_critico:.3f}; \u221E[")

Região de Aceitação (RA): [0; 6.635[
Região Crítica (RC): [6.635; ∞[


<font color="blue">
    
### 2.4.3. Risco relativo (RR) 
</font>

<font color="blue">

#### e) A intensidade da associação entre as duas variáveis recorrendo por um lado ao risco relativo e por outro analisando a sua discrepância para o odds rácio (OR). </font>


In [10]:
calcular_odds_ratio_razao_risco_discrepancia(tabela_contingencia, print_results=True)

Value 95% CI Lower 95% CI Upper
Odds Ratio for Sexo (1. Mulher / 2. Homem)   5.357        4.604        6.233
RR (ou RP) for Comportamento = 1. Agressivo  2.329        2.145        2.527
RR (ou RP) for Comportamento = 2. Assertivo  0.435        0.401        0.471
N. of Valid Cases                             3182

Discrepância = 1.301 e (θ * p21) = 1.599
